# Texture Classification Using GLCM and Rotation-Invariant LBP

This notebook compares two classical texture descriptors for image classification:

- **Gray-Level Co-occurrence Matrix (GLCM)** features
- **Rotation-invariant Local Binary Patterns (LBP + VAR)**

Both descriptors are evaluated using a **K-Nearest Neighbours (KNN)** classifier on a multi-class texture dataset containing rotated test samples.

The goal is to examine:
- Sensitivity of texture descriptors to rotation
- Feature robustness under noise and local variation
- Failure modes revealed through confusion matrices


## Texture Feature Extraction

Two complementary texture descriptors are evaluated:

### Gray-Level Co-occurrence Matrix (GLCM)
GLCM features capture second-order pixel intensity statistics by measuring how often pixel pairs with specific intensity values occur at fixed spatial offsets.  
Multiple distances and orientations are used to encode texture structure across directions.

The following properties are extracted:
- Contrast
- Energy
- Homogeneity
- Correlation

### Rotation-Invariant Local Binary Patterns (LBP)
Rotation-invariant uniform LBP encodes local texture structure by thresholding pixel neighborhoods and constructing a histogram of binary patterns.

Additionally, the **LBP variance (VAR)** descriptor captures local contrast information.  
LBP and VAR histograms are concatenated to form a single feature vector per image.


In [ ]:
import skimage.io as io
import skimage.util as util
import skimage.segmentation as seg
import matplotlib.pyplot as plt
import skimage.morphology as morph
import scipy.ndimage as ndi
import numpy as np
import skimage.color as color

import skimage.io as io
import os as os
import pandas as pd
import skimage.feature as feat
# we need to cal GLCM and also LBP 

#glcm
    # convert to grayscale
    # cal glcm for 4 dirs
    #extract festures
    # normalize?

#lbp
    #use the function
    #make hists
    #join hists




training_path = './brodatztraining/brodatztraining/'

files = pd.read_csv('./brodatztraining/brodatztraining.csv', header=None)[0].tolist()
#files = files[0:1]
print(len(files))



glcmTrain=[]
lbpTrain =[]

for i, filename in enumerate(files):
    if not filename.endswith('.png'):
        continue


    img = io.imread(os.path.join(training_path, filename))
    glcm = feat.graycomatrix(img , distances=[1,2,3,4],angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True,normed=True )
    #print(glcm)


    #extract features
    contrast= feat.graycoprops(glcm, 'contrast').flatten()
    energy = feat.graycoprops(glcm, 'energy').flatten()
    homogeneity= feat.graycoprops(glcm, 'homogeneity').flatten()
    correlation = feat.graycoprops(glcm, 'correlation').flatten()


    features = np.concatenate([contrast,energy,homogeneity,correlation])
    #features = features/np.linalg.norm(features)
    glcmTrain.append(features)



#glcmList
# lbp
    lbp = feat.local_binary_pattern(img, P=8,R=1, method='uniform')

#10 bin hist
    lbpHist, _ = np.histogram(lbp,bins=10,range=(0,10) )
    #norm
    lbpHist = lbpHist / np.sum(lbpHist)

#lbp var
    lbpVAR = feat.local_binary_pattern(img, P=8,R=1, method='var')
    lbpVARHist, _ = np.histogram(lbpVAR,bins=16,range=(0,7000) )
    lbpVARHist = lbpVARHist/np.sum(lbpVARHist)

    #concat
    lbpfeatvec = np.concatenate([lbpHist,lbpVARHist])
    lbpTrain.append(lbpfeatvec)



glcmTrainFeatures = np.array(glcmTrain)   #this would be 120 imgs by 4 features
lbpTrainFeatures = np.array(lbpTrain)     # this is 120 imgs by 26 features

print(glcmTrainFeatures.shape)
print(lbpTrainFeatures.shape)


120
(120, 64)
(120, 26)


## Test Feature Extraction

The same GLCM and LBP feature extraction pipeline is applied to the test dataset.  
Test images include arbitrary rotations, allowing evaluation of descriptor robustness to orientation changes.


In [ ]:
test_path = './brodatztesting/brodatztesting/'

files = pd.read_csv('./brodatztesting/brodatztesting.csv', header=None)[0].tolist()
#files = files[0:1]
print(len(files))


glcmTest=[]
lbpTest =[]

for i, filename in enumerate(files):
    if not filename.endswith('.png'):
        continue


    img = io.imread(os.path.join(test_path, filename))
    glcm = feat.graycomatrix(img , distances=[1,2,3,4],angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True,normed=True )
    #print(glcm)


    #extract features
    contrast= feat.graycoprops(glcm, 'contrast').flatten()
    energy = feat.graycoprops(glcm, 'energy').flatten()
    homogeneity= feat.graycoprops(glcm, 'homogeneity').flatten()
    correlation = feat.graycoprops(glcm, 'correlation').flatten()
    #asm = feat.graycoprops(glcm, 'asm').mean()

    #features = [contrast,energy,homogeneity,correlation]
    features = np.concatenate([contrast,energy,homogeneity,correlation])
    glcmTest.append(features)


#glcmList
# lbp
    lbp = feat.local_binary_pattern(img, P=8,R=1, method='uniform')

#10 bin hist
    lbpHist, _ = np.histogram(lbp,bins=10,range=(0,10) )
    #norm
    lbpHist = lbpHist / np.sum(lbpHist)

#lbp var
    lbpVAR = feat.local_binary_pattern(img, P=8,R=1, method='var')
    lbpVARHist, _ = np.histogram(lbpVAR,bins=16,range=(0,7000) )
    lbpVARHist = lbpVARHist/np.sum(lbpVARHist)

    #concat
    lbpfeatvec = np.concatenate([lbpHist,lbpVARHist])
    lbpTest.append(lbpfeatvec)



glcmTestFeatures = np.array(glcmTest)   #this would be 320 imgs by 4 features
lbpTestFeatures = np.array(lbpTest)     # this is 320 imgs by 26 features

print(glcmTestFeatures.shape)
print(lbpTestFeatures.shape)


320
(320, 64)
(320, 26)


## Ground Truth Labels

Class labels are generated based on dataset ordering, with each texture class represented by an equal number of samples in both training and test sets.


In [ ]:

# these are the truth class labels for the test & train set
yTrain = np.repeat(np.arange(1,9),15)

# will compare y pred to ytest to cal correctness
yTest = np.repeat(np.arange(1,9),40)



## KNN Classification

Separate KNN classifiers are trained for:
- GLCM feature vectors
- LBP + VAR feature vectors

This enables a direct comparison of descriptor effectiveness under identical classification conditions.


In [ ]:
import sklearn.neighbors as knn


### knn GLCM

glcmKnn = knn.KNeighborsClassifier(n_neighbors=3)
glcmKnn.fit(glcmTrainFeatures,yTrain)

### knn LBP

lbpKnn = knn.KNeighborsClassifier(n_neighbors=3)
lbpKnn.fit(lbpTrainFeatures,yTrain)




,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


# Predict the classes of the test images

Predict the classes of the test images using both classifiers.

In [ ]:
import sklearn.metrics as met


### knn GLCM
glcmPreds = glcmKnn.predict(glcmTestFeatures)
glcmCM = met.confusion_matrix(yTest, glcmPreds)


### knn LBP
lbpPreds = lbpKnn.predict(lbpTestFeatures)
lbpCM = met.confusion_matrix(yTest, lbpPreds)

## Classification Results

Classifier performance is evaluated using:
- Confusion matrices
- Overall classification accuracy
- Inspection of misclassified samples


In [ ]:

print("GLCM Confusion matrix: ")
print(glcmCM)

print("Incorrectly classified images:")
for i in range(len(yTest)):
    if glcmPreds[i] != yTest[i]:
        print(files[i], "::: predicted:", glcmPreds[i], "expected:", yTest[i])
accuracy = np.mean(glcmPreds == yTest)
print("Correct Classificaiton Rate: "+str(accuracy*100) + "%")

print("")
print("")

print("LBP Confusion matrix: ")
print(lbpCM)

print("Incorrectly classified images:")
for i in range(len(yTest)):
    if lbpPreds[i] != yTest[i]:
        print(files[i], "::: predicted:", lbpPreds[i], "expected:", yTest[i])

accuracy = np.mean(lbpPreds == yTest)
print("Correct Classificaiton Rate: "+str(accuracy*100) + "%")



GLCM Confusion matrix: 
[[27  1 10  0  0  0  2  0]
 [12 28  0  0  0  0  0  0]
 [18  4 18  0  0  0  0  0]
 [ 1  0  0 27  0  0 12  0]
 [ 0  0  0  0 40  0  0  0]
 [ 0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  8 32  0]
 [ 0  0  0  0  0  0  0 40]]
Incorrectly classified images:
patch-116291.png ::: predicted: 3 expected: 1
patch-120541.png ::: predicted: 2 expected: 1
patch-124729.png ::: predicted: 3 expected: 1
patch-131612.png ::: predicted: 3 expected: 1
patch-131973.png ::: predicted: 3 expected: 1
patch-136304.png ::: predicted: 7 expected: 1
patch-140252.png ::: predicted: 3 expected: 1
patch-140870.png ::: predicted: 3 expected: 1
patch-142400.png ::: predicted: 7 expected: 1
patch-144596.png ::: predicted: 3 expected: 1
patch-158692.png ::: predicted: 3 expected: 1
patch-159392.png ::: predicted: 3 expected: 1
patch-163446.png ::: predicted: 3 expected: 1
patch-204094.png ::: predicted: 1 expected: 2
patch-205575.png ::: predicted: 1 expected: 2
patch-206787.png ::: predicted: 1 expe

## Comparative Analysis and Error Discussion

### GLCM vs LBP Performance

GLCM-based classification performed significantly worse than LBP-based classification.  
Although multiple distances and orientations were used, GLCM features remain highly sensitive to rotation and noise.  
Concatenating statistics across offsets increases feature dimensionality and amplifies variance, which degrades KNN performance.

In contrast, rotation-invariant LBP achieved near-perfect classification accuracy.  
LBP encodes local texture structure rather than global co-occurrence statistics, allowing it to preserve texture identity under arbitrary rotations.

### Misclassification Patterns

For GLCM, the most frequent confusions occurred between texture classes with similar grayscale co-occurrence statistics, particularly where global intensity distributions were alike.

LBP misclassifications were rare and limited to texture pairs with very low contrast or high local noise, resulting in similar local binary pattern histograms.  
These cases fall close together in feature space, leading to occasional classification ambiguity.
